In [1]:
# made by Gibeom LEE, HI LAB
# 한 폴더안에 모든 csv파일의 맨 처음부터 21번째 행까지를 지웁니다. (오실로스코프 정보)
# 또한, 시작하는 시간을 0초 부터 시작하게 합니다.
# 이미 처리된 파일일 경우, 작동하지 않습니다.

In [2]:
import pandas as pd
import numpy as np
import os
import glob  # 폴더 내 파일 리스트 가져오는 라이브러리

In [3]:
# 파일이 있는 폴더 경로
folder_path = r"C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\1. ionic DC-TENG\0. 실험자료\6. data measurement\250921\csv파일"


In [4]:
# 해당 폴더 내 모든 CSV 파일 가져오기
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

for file_path in csv_files:
    try:
        # 파일명에서 공백이 있으면 _로 대체
        dir_name, file_name = os.path.split(file_path)
        new_file_name = file_name.replace(' ', '_')
        new_file_path = os.path.join(dir_name, new_file_name)

        # 파일명 변경 (공백이 있을 경우에만)
        if file_name != new_file_name:
            os.rename(file_path, new_file_path)
            print(f"Renamed file: {file_name} -> {new_file_name}")
            file_path = new_file_path  # 경로 업데이트

        # ---- 안전장치: 처음 30줄 확인 ----
        preview = pd.read_csv(file_path, header=None, nrows=30)
        contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()

        if not contains_text:
            print(f"Skipped (이미 처리된 파일입니다.): {file_path}")
            continue
        # ----------------------------------

        # Pandas로 21줄 스킵해서 로드
        df = pd.read_csv(file_path, header=None, skiprows=21)

        # 숫자 변환 + NaN 제거
        df = df.apply(pd.to_numeric, errors='coerce').dropna()

        if df.empty:
            print(f"No valid data in {file_path}")
            continue

        # NumPy 변환
        data = df.values

        # 시간 재계산
        num_rows = data.shape[0]
        if num_rows > 1:
            start_time = data[0, 0]
            end_time = data[-1, 0]
            time_interval = (end_time - start_time) / (num_rows - 1)
            time_col = np.arange(num_rows) * time_interval
        else:
            time_col = np.array([0.0])

        # 새 데이터 생성
        new_data = np.column_stack((time_col, data[:, 1]))

        # 파일 저장 (덮어쓰기)
        np.savetxt(file_path, new_data, delimiter=',', fmt='%.10f')

        print(f"Filtered data saved at: {file_path}")

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

Renamed file: 0921 10 ac 2.csv -> 0921_10_ac_2.csv
Filtered data saved at: C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\1. ionic DC-TENG\0. 실험자료\6. data measurement\250921\csv파일\0921_10_ac_2.csv
Renamed file: 0921 10 ac.csv -> 0921_10_ac.csv


C:\Users\user\AppData\Local\Temp\ipykernel_25432\3065148802.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()
C:\Users\user\AppData\Local\Temp\ipykernel_25432\3065148802.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()


Filtered data saved at: C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\1. ionic DC-TENG\0. 실험자료\6. data measurement\250921\csv파일\0921_10_ac.csv
Renamed file: 0921 10 dc 2.csv -> 0921_10_dc_2.csv
Filtered data saved at: C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\1. ionic DC-TENG\0. 실험자료\6. data measurement\250921\csv파일\0921_10_dc_2.csv
Renamed file: 0921 10 dc.csv -> 0921_10_dc.csv


C:\Users\user\AppData\Local\Temp\ipykernel_25432\3065148802.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()
C:\Users\user\AppData\Local\Temp\ipykernel_25432\3065148802.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()


Filtered data saved at: C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\1. ionic DC-TENG\0. 실험자료\6. data measurement\250921\csv파일\0921_10_dc.csv
Renamed file: 0921 12 ac 2.csv -> 0921_12_ac_2.csv
Filtered data saved at: C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\1. ionic DC-TENG\0. 실험자료\6. data measurement\250921\csv파일\0921_12_ac_2.csv
Renamed file: 0921 12 ac.csv -> 0921_12_ac.csv


...

Filtered data saved at: C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\1. ionic DC-TENG\0. 실험자료\6. data measurement\250921\csv파일\0921_pi2.csv
Renamed file: 0921 pi3.csv -> 0921_pi3.csv
Filtered data saved at: C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\1. ionic DC-TENG\0. 실험자료\6. data measurement\250921\csv파일\0921_pi3.csv
Renamed file: 0921 ptfe1.csv -> 0921_ptfe1.csv


C:\Users\user\AppData\Local\Temp\ipykernel_25432\3065148802.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()
C:\Users\user\AppData\Local\Temp\ipykernel_25432\3065148802.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()


Filtered data saved at: C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\1. ionic DC-TENG\0. 실험자료\6. data measurement\250921\csv파일\0921_ptfe1.csv
Renamed file: 0921 ptfe2.csv -> 0921_ptfe2.csv
Filtered data saved at: C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\1. ionic DC-TENG\0. 실험자료\6. data measurement\250921\csv파일\0921_ptfe2.csv
Renamed file: 0921 ptfe3.csv -> 0921_ptfe3.csv


C:\Users\user\AppData\Local\Temp\ipykernel_25432\3065148802.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()
C:\Users\user\AppData\Local\Temp\ipykernel_25432\3065148802.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_text = preview.applymap(lambda x: isinstance(x, str)).any().any()


Filtered data saved at: C:\Users\user\Desktop\Drive파일\HI Lab\0. Projects\0. On going\1. ionic DC-TENG\0. 실험자료\6. data measurement\250921\csv파일\0921_ptfe3.csv
